In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('output.csv')

In [3]:
df.head()

,Reviews,Sentiments
0,the dresser is a small but absolutely wonder...,1
1,this is a treat for fans of z grade movies he...,0
2,i thought hugh o conor was astounding as the y...,1
3,like the other guy said it sux you can count...,0
4,29 sept 1990 marked a small but important mile...,1


In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
tfidf = TfidfVectorizer(stop_words='english')

param_grid = {'logisticregression__C':[0.001,0.01,0.1,1.0,10.0,100.0]}

In [62]:
#lr = LogisticRegression()

In [6]:
pip_tfidf = make_pipeline(tfidf,LogisticRegression())
pip_tfidf.steps

[('tfidfvectorizer',
  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words='english', strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('logisticregression',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]

In [7]:
X = df.loc[:5000,'Reviews'].values
y = df.loc[:5000,'Sentiments'].values

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)

In [9]:
grid_lr_tfidf = GridSearchCV(pip_tfidf,param_grid,scoring='accuracy')

In [10]:
grid_lr_tfidf.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_i...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'logisticregression__C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [11]:
clf = grid_lr_tfidf.best_estimator_

In [13]:
clf.score(X_test,y_test)

0.86570743405275774

In [14]:
clf.score(X_train,y_train)

1.0

In [15]:
grid_lr_tfidf.best_params_

{'logisticregression__C': 10.0}

In [27]:
#SVM & PCA
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [28]:
pca = PCA(n_components=150,svd_solver='randomized',random_state=0)
svc = SVC(kernel='linear',class_weight='balanced')
pipe = make_pipeline(tfidf,svc)

In [29]:
param_grid = {'svc__C':[1,5,10,50],
             'svc__gamma':[0.0001, 0.0005, 0.001, 0.005]}

In [30]:
grid_svc = GridSearchCV(pipe,param_grid)

In [31]:
grid_svc.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_i...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'svc__C': [1, 5, 10, 50], 'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [40]:
clf1 = grid_svc.best_estimator_

In [32]:
X_test[0]

'we start all of our reviews with the following information  my wife and i have seen nearly 100 movies per year for the past 15 years  recently  we were honored by receiving lifetime movie passes to any movie any time at no cost  so we can see whatever we want whenever we want  the point of this is that critics count for zero  your local critics or the national critics like ebert are really no different than you or me  the only difference is that they get to write about the movie and are forced to see hundreds of movies whether they want to or not therefore  it is our belief that if you get your monies worth for two hours of enjoyment that is good enough for us  we never ever listen or read the critics  we only care about our friends and those who we know like the same things as us  well enough about that   br    br   when meryl streep the head of the nsc in the movie says  the united states does not torture  it got a big laugh at this movie  it is of course a lie that the bush adminis

In [33]:
type(y_test[0])

numpy.int64

In [41]:
clf1.score(X_test,y_test)

0.8689048760991207

In [42]:
clf1.score(X_train,y_train)

0.99093333333333333

#Naive Bayes

In [43]:
from sklearn.naive_bayes import MultinomialNB

In [44]:
pipe1 = make_pipeline(tfidf,MultinomialNB())

In [45]:
pipe1.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_i...   vocabulary=None)), ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [46]:
pipe1.score(X_test,y_test)

0.85131894484412474

In [47]:
pipe1.score(X_train,y_train)

0.96799999999999997

In [48]:
from sklearn.metrics import confusion_matrix

In [50]:
pred = pipe1.predict(X_test)

In [51]:
cm = confusion_matrix(y_test,pred)

In [52]:
cm

array([[527,  89],
       [ 97, 538]], dtype=int64)